# 1. Introduction

<div style="color:white;display:fill;
            background-color:#48AFFF;font-size:160%;
            font-family:Arial">
    <p style="padding: 4px;color:white;"><b>1.1 Context</b></p>
</div>

LightGBM implemenation for the Porto Seguro's Safe Driver Prediction competition.

<div style="color:white;display:fill;
            background-color:#48AFFF;font-size:160%;
            font-family:Arial">
    <p style="padding: 4px;color:white;"><b>1.2 Used code</b></p>
</div>

The code used is adapted from the chapter Buidling a LightGBM submission in [The Kaggle Workbook](https://www.amazon.com/Kaggle-Workbook-Self-learning-exercises-competitions/dp/1804611212) by [(Banachewicz & Massaron)](#5.-References) and directly from the workbook [workbook_lgb](https://www.kaggle.com/code/lucamassaron/workbook-lgb) by [Luca Massaron](https://www.kaggle.com/lucamassaron).

<div style="color:white;display:fill;
            background-color:#48AFFF;font-size:160%;
            font-family:Arial">
    <p style="padding: 4px;color:white;"><b>1.3 Libraries</b></p>
</div>


In [1]:
import numpy as np
import pandas as pd
import optuna
import lightgbm as lgb

from numba import jit
from path import Path
from sklearn.model_selection import StratifiedKFold

# 2. Data Processing

<div style="color:white;display:fill;
            background-color:#48AFFF;font-size:160%;
            font-family:Arial">
    <p style="padding: 4px;color:white;"><b>2.1 Configuration Class</b></p>
</div>

For ease of use and modifiability we are using a config class. This allows us to have all parameters in one place and easily change them without having to go through the code having to look for changes we missed. 

The params set here are based on the discussion post of [Michael Jahrer](https://www.kaggle.com/mjahrer) [(1st place with representation learning)](https://www.kaggle.com/competitions/porto-seguro-safe-driver-prediction/discussion/44629)

The optuna_lgb flag will come into effect in [chapter 3](#3.-Evaluation-Metric) where we define a function to perform a search with [Optuna]( https://optuna.org/) to automate our hyperparameter search. If the flag is set to False, then we will stick with the parameters proposed by [Michael Jahrer](https://www.kaggle.com/competitions/porto-seguro-safe-driver-prediction/discussion/44629).

Config_Optuna contains the values that came as a result of running the search with [Optuna]( https://optuna.org/).

In [2]:
class Config_Jahrer:
    input_path = Path('../input/porto-seguro-safe-driver-prediction')
    optuna_lgb = False
    n_estimators = 1500
    early_stopping_round = 150
    cv_folds = 5
    random_state = 0
    params = {'objective': 'binary',
              'boosting_type': 'gbdt',
              'learning_rate': 0.01,
              'max_bin': 25,
              'num_leaves': 31,
              'min_child_samples': 1500,
              'colsample_bytree': 0.7,
              'subsample_freq': 1,
              'subsample': 0.7,
              'reg_alpha': 1.0,
              'reg_lambda': 1.0,
              'verbosity': 0,
              'random_state': 0}
    
class Config_Optuna:
    input_path = Path('../input/porto-seguro-safe-driver-prediction')
    optuna_lgb = False
    n_estimators = 1500
    early_stopping_round = 150
    cv_folds = 5
    random_state = 0
    params = {'objective': 'binary',
              'boosting_type': 'gbdt',
              'learning_rate': 0.011967899898885897,
              'max_bin': 25,
              'num_leaves': 28,
              'min_child_samples': 2355,
              'colsample_bytree': 0.5423725039317776,
              'subsample_freq': 9,
              'subsample': 0.9158581442164284,
              'reg_alpha': 1.1294388302276255e-07,
              'reg_lambda': 1.823664272960681e-05,
              'verbosity': 0,
              'random_state': 0}
    
config = Config_Optuna()


<div style="color:white;display:fill;
            background-color:#48AFFF;font-size:160%;
            font-family:Arial">
    <p style="padding: 4px;color:white;"><b>2.2 Loading the data</b></p>
</div>


In [3]:
train = pd.read_csv(config.input_path / 'train.csv', index_col='id')
test = pd.read_csv(config.input_path / 'test.csv', index_col='id')
submission = pd.read_csv(config.input_path / 'sample_submission.csv', index_col='id')

In [4]:
calc_features = [feat for feat in train.columns if "_calc" in feat]
cat_features = [feat for feat in train.columns if "_cat" in feat]

In [5]:
train.head()

,target,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
id,,,,,,,,,,,,,,,,,,,,,
7,0,2,2,5,1,0,0,1,0,0,...,9,1,5,8,0,1,1,0,0,1
9,0,1,1,7,0,0,0,0,1,0,...,3,1,1,9,0,1,1,0,1,0
13,0,5,4,9,1,0,0,0,1,0,...,4,2,7,7,0,1,1,0,1,0
16,0,0,1,2,0,0,1,0,0,0,...,2,2,4,9,0,0,0,0,0,0
17,0,0,2,0,1,0,1,0,0,0,...,3,1,1,3,0,0,0,1,1,0


In [6]:
test.head()

,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
id,,,,,,,,,,,,,,,,,,,,,
0,0,1,8,1,0,0,1,0,0,0,...,1,1,1,12,0,1,1,0,0,1
1,4,2,5,1,0,0,0,0,1,0,...,2,0,3,10,0,0,1,1,0,1
2,5,1,3,0,0,0,0,0,1,0,...,4,0,2,4,0,0,0,0,0,0
3,0,1,6,0,0,1,0,0,0,0,...,5,1,0,5,1,0,1,0,0,0
4,5,1,7,0,0,0,0,0,1,0,...,4,0,0,4,0,1,1,0,0,1


In [7]:
submission.head()

,target
id,
0,0.0364
1,0.0364
2,0.0364
3,0.0364
4,0.0364


In [8]:
target = train["target"]
train = train.drop("target", axis="columns")

<div style="color:white;display:fill;
            background-color:#48AFFF;font-size:160%;
            font-family:Arial">
    <p style="padding: 4px;color:white;"><b>2.3 Dropping the calc features</b></p>
</div>

We are dropping the calc features as discussed here: [Why removing ps_calc features improve the results?](https://www.kaggle.com/competitions/porto-seguro-safe-driver-prediction/discussion/41970)

In [9]:
train = train.drop(calc_features, axis="columns")
test = test.drop(calc_features, axis="columns")

<div style="color:white;display:fill;
            background-color:#48AFFF;font-size:160%;
            font-family:Arial">
    <p style="padding: 4px;color:white;"><b>2.4 One-hot Encoding</b></p>
</div>

In [10]:
train = pd.get_dummies(train, columns=cat_features)
test = pd.get_dummies(test, columns=cat_features)

In [11]:
assert((train.columns==test.columns).all())

# 3. Evaluation Metric

<div style="color:white;display:fill;
            background-color:#48AFFF;font-size:160%;
            font-family:Arial">
    <p style="padding: 4px;color:white;"><b>3.1 Normalized Gini Coefficient</b></p>
</div>

In [12]:
@jit
def eval_gini(y_true, y_pred):
    y_true = np.asarray(y_true)
    y_true = y_true[np.argsort(y_pred)]
    ntrue = 0
    gini = 0
    delta = 0
    n = len(y_true)
    for i in range(n-1, -1, -1):
        y_i = y_true[i]
        ntrue += y_i
        gini += y_i * delta
        delta += 1 - y_i
    gini = 1 - 2 * gini / (ntrue * (n - ntrue))
    return gini

In [13]:
def gini_lgb(y_true, y_pred):
    eval_name = 'normalized_gini_coef'
    eval_result = eval_gini(y_true, y_pred)
    is_higher_better = True
    return eval_name, eval_result, is_higher_better

The training parameters as suggested by [Michael Jahrer](https://www.kaggle.com/mjahrer) in his discussion post [(1st place with representation learning)](https://www.kaggle.com/competitions/porto-seguro-safe-driver-prediction/discussion/44629) work the best.

<div style="color:white;display:fill;
            background-color:#48AFFF;font-size:160%;
            font-family:Arial">
    <p style="padding: 4px;color:white;"><b>3.2 Search by Optuna</b></p>
</div>

We perform a search with [optuna]( https://optuna.org/) to automate our hyperparameter search and assign them to config.params. If the optuna_lgb flag is set to False, we use the params set in the config class.

In [14]:
 if config.optuna_lgb:
        
    def objective(trial):
        params = {
                'learning_rate': trial.suggest_float("learning_rate", 0.01, 1.0),
                'num_leaves': trial.suggest_int("num_leaves", 3, 255),
                'min_child_samples': trial.suggest_int("min_child_samples", 3, 3000),
                'colsample_bytree': trial.suggest_float("colsample_bytree", 0.1, 1.0),
                'subsample_freq': trial.suggest_int("subsample_freq", 0, 10),
                'subsample': trial.suggest_float("subsample", 0.1, 1.0),
                'reg_alpha': trial.suggest_loguniform("reg_alpha", 1e-9, 10.0),
                'reg_lambda': trial.suggest_loguniform("reg_lambda", 1e-9, 10.0),
        }
        
        score = list()
        skf = StratifiedKFold(n_splits=config.cv_folds, shuffle=True, random_state=config.random_state)

        for train_idx, valid_idx in skf.split(train, target):
            X_train, y_train = train.iloc[train_idx], target.iloc[train_idx]
            X_valid, y_valid = train.iloc[valid_idx], target.iloc[valid_idx]

            model = lgb.LGBMClassifier(**params,
                                    n_estimators=1500,
                                    early_stopping_round=150,
                                    force_row_wise=True)

            callbacks=[lgb.early_stopping(stopping_rounds=150, verbose=False)]
            model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], eval_metric=gini_lgb, callbacks=callbacks)
            score.append(model.best_score_['valid_0']['normalized_gini_coef'])

        return np.mean(score)

    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=300)

    print("Best Gini Normalized Score", study.best_value)
    print("Best parameters", study.best_params)
    
    params = {'objective': 'binary',
            'boosting_type': 'gbdt',
            'verbosity': 0,
            'random_state': 0}
    
    params.update(study.best_params)
    
else:
    params = config.params

# 4. Training and Submission

<div style="color:white;display:fill;
            background-color:#48AFFF;font-size:160%;
            font-family:Arial">
    <p style="padding: 4px;color:white;"><b>4.1 Training</b></p>
</div>

In [15]:
preds = np.zeros(len(test))
oof = np.zeros(len(train))
metric_evaluations = list()

skf = StratifiedKFold(n_splits=config.cv_folds, shuffle=True, random_state=config.random_state)

for idx, (train_idx, valid_idx) in enumerate(skf.split(train, target)):
    print(f"CV fold {idx}")
    X_train, y_train = train.iloc[train_idx], target.iloc[train_idx]
    X_valid, y_valid = train.iloc[valid_idx], target.iloc[valid_idx]
    
    model = lgb.LGBMClassifier(**params,
                               n_estimators=config.n_estimators,
                               early_stopping_round=config.early_stopping_round,
                               force_row_wise=True)
    
    callbacks=[lgb.early_stopping(stopping_rounds=150), 
               lgb.log_evaluation(period=100, show_stdv=False)]
                                                                                           
    model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], eval_metric=gini_lgb, callbacks=callbacks)
    metric_evaluations.append(model.best_score_['valid_0']['normalized_gini_coef'])
    preds += model.predict_proba(test, num_iteration=model.best_iteration_)[:,1] / skf.n_splits
    oof[valid_idx] = model.predict_proba(X_valid, num_iteration=model.best_iteration_)[:,1]

CV fold 0
Training until validation scores don't improve for 150 rounds
[100]	valid_0's binary_logloss: 0.153063	valid_0's normalized_gini_coef: 0.273457
[200]	valid_0's binary_logloss: 0.152135	valid_0's normalized_gini_coef: 0.282336
[300]	valid_0's binary_logloss: 0.151753	valid_0's normalized_gini_coef: 0.288449
[400]	valid_0's binary_logloss: 0.151567	valid_0's normalized_gini_coef: 0.291844
[500]	valid_0's binary_logloss: 0.151472	valid_0's normalized_gini_coef: 0.293402
[600]	valid_0's binary_logloss: 0.151429	valid_0's normalized_gini_coef: 0.294205
[700]	valid_0's binary_logloss: 0.151381	valid_0's normalized_gini_coef: 0.295304
[800]	valid_0's binary_logloss: 0.151377	valid_0's normalized_gini_coef: 0.295314
[900]	valid_0's binary_logloss: 0.15138	valid_0's normalized_gini_coef: 0.295206
Early stopping, best iteration is:
[767]	valid_0's binary_logloss: 0.151374	valid_0's normalized_gini_coef: 0.295421
CV fold 1
Training until validation scores don't improve for 150 rounds
[1

In [16]:
print(f"LightGBM CV Gini Normalized Score: {np.mean(metric_evaluations):0.3f} ({np.std(metric_evaluations):0.3f})")

LightGBM CV Gini Normalized Score: 0.288 (0.016)


<div style="color:white;display:fill;
            background-color:#48AFFF;font-size:160%;
            font-family:Arial">
    <p style="padding: 4px;color:white;"><b>4.2 Submission</b></p>
</div>

Saving the submission to the required (according to the competitions evaluation page) submission.csv file.

In [17]:
submission['target'] = preds
submission.to_csv('submission.csv')

In [18]:
oofs = target.to_frame()
oofs['target'] = oof
oofs.to_csv('lgb_oof.csv')

Saving the out-of-fold predictions for later blending with other models.

# 5. References

<div style="color:white;display:fill;
            background-color:#48AFFF;font-size:160%;
            font-family:Arial">
    <p style="padding: 4px;color:white;"><b>5.1 References</b></p>
</div>

* Banachewicz, Konrad; Massaron, Luca. [The Kaggle Book](https://www.kaggle.com/general/320574): Data analysis and machine learning for competitive data science. Packt Publishing
* Banachewicz, Konrad; Massaron, Luca. [The Kaggle Workbook](https://www.amazon.com/Kaggle-Workbook-Self-learning-exercises-competitions/dp/1804611212): Self-learning exercises and valuable insights for Kaggle data science competitions. Packt Publishing. 
* [workbook_lgb](https://www.kaggle.com/code/lucamassaron/workbook-lgb) by [Luca Massaron](https://www.kaggle.com/lucamassaron)
* [Why removing ps_calc features improve the results?](https://www.kaggle.com/competitions/porto-seguro-safe-driver-prediction/discussion/41970) by [Hossein Amirkhani](https://www.kaggle.com/hatime)
* [1st place with representation learning](https://www.kaggle.com/competitions/porto-seguro-safe-driver-prediction/discussion/44629) by [Michael Jahrer](https://www.kaggle.com/mjahrer) 
* [Optuna]( https://optuna.org/) (An open source hyperparameter optimization framework to automate hyperparameter search)